In [1]:
import os, sys

sys.path.append(os.path.abspath(os.path.join('..')))

from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env.local")

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {os.getenv('TMDB_API_KEY')}",
}

In [2]:
from movie_utils import *

from utils import *

c:\Me\Duke\Classes\Summer '24\MLOps\cinelens\api\env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# titles = [
#     'Captain America: Civil War',
#     'Abigail',
#     'Mission: Impossible - Fallout',
#     'John Wick: Chapter 4',
#     'Se7en'
#     ]

# movies = [
#     get_details(title, headers)
#     for title in titles
# ]

# movies_new = []

# for movie in movies:
#     movies_new.append(
#         {
#             'title': movie['title'],
#             'overview': movie['overview'],
#             'release_date': movie['release_date'],
#             'rating': round(movie['rating'] / 2, 1),
#             'tagline': movie['tagline'],
#             'poster': movie['poster'],
#             'backdrop': movie['backdrop'],
#             'runtime': movie['runtime'],
#             'genres': (movie['genres']),
#             'providers': get_new_providers(movie['providers'])
#         }
#     )

In [4]:
import time

from concurrent.futures import ThreadPoolExecutor

In [5]:
client = connect_to_mongo()

titles = get_titles(n=20) 

query = """
A successful banker is wrongfully convicted of murdering his wife and her lover and is sentenced to life
imprisonment in a harsh penitentiary. Over the decades, he forms an unlikely friendship with a fellow inmate,
a seasoned contraband smuggler. Despite the brutality and corruption within the prison walls, he maintains
hope and dignity, using his financial skills to assist the prison staff, ultimately orchestrating a daring
and ingenious escape plan. The narrative explores themes of hope, resilience, and the enduring human spirit in the face of adversity.
"""

query = """
A movie about a guy dressed like a bat or something. I don't know, it's got Christian Bale in it.
"""

start_time = time.time()

res_text = ""
with ThreadPoolExecutor(max_workers=6) as executor:  # Adjust max_workers based on your system's capabilities
    futures = []
    futures.append(executor.submit(query_rag_movie, query, client, titles))
    
    for future in futures:
        res_text += future.result()

end_time = time.time()

elapsed_time = end_time - start_time
formatted_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

print(f"Execution time: {formatted_time}")


Pinged your deployment. You successfully connected to MongoDB!
Using existing file
Execution time: 00:01:34


In [6]:
res_text

'Return at least 5 movies that are similar to the movie described above among the following: [\'The Shawshank Redemption\', \'The Godfather\', \'The Godfather Part II\', "Schindler\'s List", \'12 Angry Men\', \'The Dark Knight\', \'The Green Mile\', \'Pulp Fiction\', \'The Lord of the Rings: The Return of the King\', \'Forrest Gump\', \'GoodFellas\', \'Fight Club\', \'Interstellar\', \'Radical\', \'Psycho\', "One Flew Over the Cuckoo\'s Nest", "Gabriel\'s Inferno", \'Once Upon a Time in America\', \'Primal: Tales of Savagery\', \'The Lord of the Rings: The Fellowship of the Ring\'] . Return just the titles of the movies in a Python list.</s>'

In [7]:
movies = []

for title in titles:
    if title.lower() in res_text.lower():
        movies.append(title)

movies

['The Shawshank Redemption',
 'The Godfather',
 'The Godfather Part II',
 "Schindler's List",
 '12 Angry Men',
 'The Dark Knight',
 'The Green Mile',
 'Pulp Fiction',
 'The Lord of the Rings: The Return of the King',
 'Forrest Gump',
 'GoodFellas',
 'Fight Club',
 'Interstellar',
 'Radical',
 'Psycho',
 "One Flew Over the Cuckoo's Nest",
 "Gabriel's Inferno",
 'Once Upon a Time in America',
 'Primal: Tales of Savagery',
 'The Lord of the Rings: The Fellowship of the Ring']

In [8]:
movies[:5]


['The Shawshank Redemption',
 'The Godfather',
 'The Godfather Part II',
 "Schindler's List",
 '12 Angry Men']